# 1. Experimentation Basics: a Standard A/B Test Example



*note: if you see any mistakes, please feel free to let me know so that I can improve the notebook!*

In [ ]:
# prompt: import numpy, pandas

import numpy as np
import pandas as pd

Knowledge on Experimentation, for certain types of Data Science, is a fairly important field to keep up with.

Especially for Product Data Science roles, the chance of being questioned on experimentation intuition during interviews is very high.

In this notebook, I want to highlight and simulate the basics of experimentation (e.g., hypothesis testing) using various different sources.

## First, an example of an A/B Test

Let's run through a very simple example of an A/B test using only freely available online-tools.

Let's say that you work for a Product team that wants more of its users to upload images with the album feature. You make some design changes that should make it easier to upload images, and the primary metric is "% of users uploading images".

Suppose that you utilize https://www.evanmiller.org/ab-testing/sample-size.html to see that you will need to run the test for 2 weeks to get 25k users into each variation (control and treatment) to be able to detect a 5% relative lift with a base rate of 20% (with power of 80% and $\alpha$=0.05%).

Let's say that you run your test and you end up with the following:

**Control:**
- num. users who uploaded images: 5020
- num. total pool of users who entered control: 25101

**Treatment:**
- num. users who uploaded images: 5201
- num. total pool of users who entered control: 25104

Then let's say you go to https://www.surveymonkey.com/mp/ab-testing-significance-calculator/, fill out the results above of a two-sided 95% test.

The website tells you that
- control: 20%
- treatment: 20.72%
- treatment was higher than control by 3.59%
- power: 80.92%
- p-value: 0.0228

Thus, since your p-value was below the pre-set alpha, you reject the null hypothesis and ship the feature.

## But what just happened?

What actually happened in the above? What is the mechanism and theory are we using to just allow a feature to ship?

Covering the basics of all that is the purpose of this notebook. We'll manually calculate everything so that it's super clear.

## A quick background of Statistical Hypothesis Testing

A statistical hypothesis test is a method of statistical inference used to decide whether the data at hand sufficiently support a particular hypothesis. Hypothesis testing allows us to make probabilistic statements about population parameters. More informally, hypothesis testing is the processes of making decisions under uncertainty ([Wikipedia](https://en.wikipedia.org/wiki/Statistical_hypothesis_testing))

All hypothesis tests use what's called a stochastic proof by contradiction. A proof by contradiction means that you prove something by first assuming the opposite of what you wish to prove and showing how this assumption leads to a contradiction.

One of the famous examples of a proof by contradiction involves proving that $\sqrt{2}$ is an irrational number by contradiction. It first involves assuming that $\sqrt{2}$ *is* a rational number, and using the properties of rational numbers it is shown that this assumption is not possible.

In a hypothesis test, it is *stochastic* because it involves uncertainty. This means that we can never conclude with certainty that a hypothesis is incorrect. Instead, we argue that the hypothesis is *likely* incorrect.

To go about this proof, we have the following 6 steps:

1. Choose $H_0$ and also $H_1$.
2. Choose a threshold $\alpha$, the maximum probability of *Type I error* one is willing to tolerate. Remember that Type I error is the probability of *falsely* rejecting the null. So thinking that there is an effect when there isn't.
3. Choose a test statistic, which is a function of the observed data
4. Derive a distribution of the test statistic under the null hypothesis. This distribution is often called the *reference distribution*.
5. Compute the p-value by comparing the observed value of the test statistic against its reference distribution.
6. Reject the null hypothesis if the p-value is less than the pre-specified threshold $\alpha$ and retain the null hypothesis otherwise.

The difficult part is knowing which test statistic is suitable for each kind of problem. It can be difficult because you have to know how to manipulate your data and assumptions to turn it into some distribution that we are familiar with on a testing level. The below will demonstrate an example using a test statistic commonly used in A/B tests.

## Two-Proportion $Z$-test Overview

When you have a proportion that you want to compare between treatment ($p_1$) and control ($p_2$), we can use the two-proportion z-test.

1. $H_0:p_1=p_2$ and $H_1:p_1\neq p_2$
2. choose $\alpha$ (usually 0.05)
3. the test statistic, a function of the observed data, for a two-proportion z-test is defined as
$$z=\frac{\hat p_1-\hat p_2}{\sqrt{\hat p(1-\hat p)\left(\frac{1}{n_1}+\frac{1}{n_2}\right)}}$$, where $\hat p=\frac{n_1\hat p_1+n_2\hat p_2}{n_1+n_2}$, which you can think of as the average rate combining both control and treatment.
4. The reference distribution is a standard normal $\mathcal{N}(0,1)$.
5. We can compute the $p$-value using the formula $p=2\times\Phi(Z\geq|z|)$ where $z$ is the observed value of the test statistic $Z$ and $\Phi(\cdot)$ is the distribution function of the standard normal random variable.
6. Reject the null if the $p$-value is less than the pre-specified $\alpha$.



## Two-Proportion $Z$-test using our A/B test example



Remember that in our example above,

Control:
- $\hat p_2 = 5020/25101\approx 19.99\%$
- $n_2=25101$

Treatment:
- $\hat p_1 = 5201/25104\approx 20.71\%$
- $n_1=25104$

In [ ]:
# inputs
p2 = 5020/25101
n2 = 25101

p1 = 5201/25104
n1 = 25104

# helper variables
p_pool = (n1*p1 + n2*p2)/(n1+n2)

# test statistic
z = (p1 - p2)/np.sqrt((p_pool*(1-p_pool))*(1/n1 + 1/n2))

In [ ]:
# prompt: compute the two-sided p-value using the test statistic variable "z" by doing 2*(1- phi(|z|)) where phi() is the cdf of the standard normal

import scipy.stats as st
p_value = 2*(1 - st.norm.cdf(abs(z)))
print(p_value)

0.04556776780769134


You'll notice that our $p$-value, $\approx 0.045$, is exactly double of the result from the SurveyMonkey site we used.

If we had decided to do a one-sided test, that would be saying that we don't even consider the possibility that the test statistic can be negative (or positive). In that case, $p=\Phi(Z\geq Z_{\text{obs}})$.

Thus, when $z$ is positive, the one-sided $p$-value is smaller than the two-sided $p$-value. Thus, given the same threshold, the one-sided test is more likely to reject the null hypothesis than the two-sided test.

Interestingly, in the SurveyMonkey site, choosing two-sided or one-sided does not change the $p$-value.

At the very least, we know how to exactly replicate the result from SurveyMonkey.


## Two-Proportion $Z$-test Math

The next natural question is, how does the math work? Why are we able to use a normal distribution to test our hypothesis?

First, what needs to be noted is that users in control and treatment each can be represented with a Bernoulli distribution by design.

A [bernoulli distribution](https://en.wikipedia.org/wiki/Bernoulli_distribution) is the discrete probability distribution of a random variable which takes the value $1$ with probability $p$ and the value $0$ with probability $q=1-p$. Then, the [binomial distribution](https://en.wikipedia.org/wiki/Binomial_distribution) models the distribution of $n$ events that independently have a $p$ chance of taking a value of $1$.


From the A/B test perspective from the example earlier, for a ratio test, we can think of it as each user in control having a $p_2$ chance of uploading images based on the current product design and engineering. We would also have another random variable for treatment users with a $p_1$ chance and what we are trying to see is if that innate rate goes up. Statistically speaking, the key here is the independence assumption. We are saying that a user uploading an image doesn't influence other people to upload or not upload images too. How reasonable this assumption is depends on the A/B test problem, often times it's fairly reasonable (such as our example, unless we discover that there's a large social aspect to image uploading), while others it is not reasonable at all. How A/B testing design is tackled in those scenarios is something we will cover in the future.

More formally speaking, this means we have two random variables:
- **Control**: random variable $X_2$ follows the binomial distribution with parameters $n_2\in\mathbb{N}$ and $p_2\in\left[0,1\right]$, written as $X_2\sim\text{Binomial}(n_2,p_2)$
- **Treatment**: random variable $X_1$ follows the binomial distribution with parameters $n_1\in\mathbb{N}$ and $p_1\in\left[0,1\right]$, written as $X_1\sim\text{Binomial}(n_1,p_1)$


The behavior of the binomial distribution is very well understood. For example, the mean of a binomial random variable $X$ is $\mathbb{E}[X]=np$ and the variance is $\text{Var}(X)=np(1-p)$.

From here, there are several important understood properties that close the bridge between this and what we were doing above with the normal distribution.

First, as $n$ gets larger, we start to be able to approximate the binomial distribution with a normal distribution. More formally, a reasonable approximation of $\text{Binomial}(n,p)$ becomes
 $$\mathcal{N}(np,np(1-p))$$

You will notice that the inputs mean and variance into the normal distribution function is simply the mean and variance of the binomial distribution. There are various rules of thumb regarding how large $n$ has to be - in many cases, especially when working with A/B tests, $n$ is more than large enough. In our case, $n_2=25101$ and $n_1=25104$.

And a quick note - we aren't actually interested in the difference in the number of successes, we want to see the rate. Thus, we actually divide $n_1$ and $n_2$ respectively so that what we're really approximating with the normal distribution are the following:
- **Control**: $\mathcal{N}(\hat p_2,\frac{\hat p_2(1-\hat p_2)}{n_2})$
- **Treatment**: $\mathcal{N}(\hat p_1,\frac{\hat p_1(1-\hat p_1)}{n_1})$

If you're wondering why the $n_i$ disappeared in the mean but not the variance, this is because of the following property:
$$\text{Var}\left(\frac{X}{n}\right)=\frac{\text{Var}(X)}{n^2}=\frac{np(1-p)}{n^2}$$

Next is a property that shows that when two random variables $X,Y$ are each normally distributed and independent, $Z=X-Y$ is also a normal distribution with $\mathbb{E}[Z]=\mu_x-\mu_y$ and $\text{Var}(Z)=\sigma_x^2+\sigma_y^2$ (with the standard deviation just being $\sqrt{\text{Var}(Z)}$. You can read more about it [here](https://mathworld.wolfram.com/NormalDifferenceDistribution.html) and [here](https://online.stat.psu.edu/stat500/book/export/html/572).

What this implies is that for our example above, $X_1-X_2$ can be shown as a random variable approximated with
$$\mathcal{N}\left(\hat p_1-\hat p_2,\frac{\hat p_1(1-\hat p_1)}{n_1}+\frac{\hat p_2(1-\hat p_2)}{n_2}\right)$$

Since we are assuming that the null is true where $\hat p_1 = \hat p_2$, we can use the pooled estimate of our variance

$$\hat p(1-\hat p)\left(\frac{1}{n_1}+\frac{1}{n_2}\right)$$, where $\hat p=\frac{n_1\hat p_1+n_2\hat p_2}{n_1+n_2}$, which is equivalent to the previous form when the null is true.

Then, we also know have the property that if a random variable $X$ follows $X\xrightarrow{d}\mathcal{N}(\mu,\sigma^2)$, then $Z=\frac{X-\mu}{\sigma}\xrightarrow{d}\mathcal{N}(0,1)$.

Hence,

$$z=\frac{(\hat p_1-\hat p_2)-0}{\sqrt{\hat p(1-\hat p)\left(\frac{1}{n_1}+\frac{1}{n_2}\right)}}$$, where $\hat p=\frac{n_1\hat p_1+n_2\hat p_2}{n_1+n_2}$.

From here on, we just We can compute the $p$-value using the formula $p=2\times\Phi(Z\geq|z|)$ (assuming a two-sided test) where $z$ is the observed value of the test statistic $Z$ and $\Phi(\cdot)$ is the distribution function of the standard normal random variable.

Then, we reject the null if the $p$-value is less than the pre-specified $\alpha$.


## Conclusion

There's still a lot to cover - for example, there were a few concepts from the A/B test example where the math was not explained - such as the power analysis (and the power calculation). In addition, other concepts like confidence intervals would benefit the intuition behind hypothesis testing as well.

Since this notebook is getting a little long, we will split other concepts in other notebooks.